In [1]:
import requests
import pandas as pd
import re
from datetime import datetime

# URL
traffic_url = "http://datamall2.mytransport.sg/ltaodataservice/TrafficIncidents"
api_key = open("api_key.txt").read()
api_headers = {"AccountKey": api_key, "accept" : "application/json"}

# Pulling Data
request = requests.get(url=traffic_url, headers=api_headers)
print(request)
data = request.json()

# Time References
dt = datetime.now()

dt_year = dt.year
dt_hour = dt.hour

<Response [200]>


In [208]:
df = pd.DataFrame(data["value"])
df

,Type,Latitude,Longitude,Message
0,Accident,1.265894,103.824578,(28/12)13:38 Accident on West Coast Highway (t...
1,Roadwork,1.326368,103.926893,(28/12)13:29 Roadworks on Bedok North Avenue 1...
2,Accident,1.317371,103.709988,(28/12)13:15 Accident on Jurong Pier Road (tow...
3,Heavy Traffic,1.301734,103.834888,(28/12)13:13 Heavy Traffic on Orchard Turn (to...
4,Roadwork,1.372522,103.908729,(28/12)13:11 Roadworks on KPE (towards ECP) be...
...,...,...,...,...
126,Roadwork,1.393270,103.775152,(28/12)9:35 Roadworks on BKE (towards PIE) at ...
127,Roadwork,1.299759,103.863671,(28/12)9:21 Roadworks on Republic Avenue (towa...
128,Roadwork,1.299830,103.832773,(28/12)9:19 Roadworks on Grange Road (towards ...
129,Roadwork,1.312927,103.798355,(28/12)9:14 Roadworks on Holland Road (towards...


In [213]:
len(df) # Add line to break

if (len(df) > 0):
    print()
    # Continue with code

In [214]:
# Extract Date and Time out of the Message to keep the main message only

date_regex = "([0-9]{1,2}/[0-9]{1,2})"
df["Date"] = df["Message"].str.extract(pat = date_regex) + "/" + str(dt.year)

time_regex = "([0-9]{1,2}:[0-9]{1,2})"
df["Time"] = df["Message"].str.extract(pat = time_regex)

message_regex = "\d\s(.*$)"
df["Message"] = df["Message"].str.extract(pat = message_regex)

In [215]:
df

,Type,Latitude,Longitude,Message,Date,Time
0,Accident,1.265894,103.824578,Accident on West Coast Highway (towards City) ...,28/12/2022,13:38
1,Roadwork,1.326368,103.926893,Roadworks on Bedok North Avenue 1 (towards Cha...,28/12/2022,13:29
2,Accident,1.317371,103.709988,Accident on Jurong Pier Road (towards Jurong P...,28/12/2022,13:15
3,Heavy Traffic,1.301734,103.834888,Heavy Traffic on Orchard Turn (towards Orchard...,28/12/2022,13:13
4,Roadwork,1.372522,103.908729,Roadworks on KPE (towards ECP) between Buangko...,28/12/2022,13:11
...,...,...,...,...,...,...
126,Roadwork,1.393270,103.775152,Roadworks on BKE (towards PIE) at KJE(PIE) Exit.,28/12/2022,9:35
127,Roadwork,1.299759,103.863671,Roadworks on Republic Avenue (towards Nicoll H...,28/12/2022,9:21
128,Roadwork,1.299830,103.832773,Roadworks on Grange Road (towards Leonie Hill ...,28/12/2022,9:19
129,Roadwork,1.312927,103.798355,Roadworks on Holland Road (towards Belmont Roa...,28/12/2022,9:14


In [217]:
# Previous Hour
if dt_hour >= 1:
    prev_hour = str(dt_hour - 1)
    df_boolean = df["Time"].str.match(pat = (prev_hour + ":"))
else:
    prev_hour = "23"
    df_boolean = df["Time"].str.match(pat = "23:")

In [219]:
df_tocloud = df[df_boolean].reset_index(drop = True)
df_tocloud

,Type,Latitude,Longitude,Message,Date,Time
0,Roadwork,1.318049,103.747868,Roadworks on West Coast Road (towards Jurong T...,28/12/2022,12:13
1,Roadwork,1.342801,103.746604,Roadworks on PIE (towards Tuas) after Toh Guan...,28/12/2022,12:12
2,Roadwork,1.344798,103.742646,Roadworks on PIE (towards Changi Airport) afte...,28/12/2022,12:12
3,Roadwork,1.356842,103.715217,Roadworks on PIE (towards Changi Airport) afte...,28/12/2022,12:11
4,Roadwork,1.322437,103.675290,Roadworks on AYE (towards Tuas) after Benoi Rd...,28/12/2022,12:10
5,Roadwork,1.272196,103.804839,Roadworks on Telok Blangah Road (towards Tuas)...,28/12/2022,12:08
6,Roadwork,1.270326,103.812437,Roadworks on Telok Blangah Road (towards City)...,28/12/2022,12:07
7,Roadwork,1.273153,103.800331,Roadworks on Pasir Panjang Road (towards Tuas)...,28/12/2022,12:05
8,Roadwork,1.361411,103.851033,Roadworks on Ang Mo Kio Avenue 1 (towards Bish...,28/12/2022,12:05
9,Roadwork,1.301004,103.786375,Roadworks on North Buona Vista Road (towards P...,28/12/2022,12:05


In [222]:
df_tocloud_key = str(dt.date()) + " " + prev_hour
df_tocloud_key

'2022-12-28 12'

## For Testing and Subsequent Wring

In [130]:
# Converting to JSON for Writing to Cloud
df_tocloud = df_tocloud.to_json()


# Reading from Cloud
import json
df_raw = json.loads(df_tocloud)
df_local = pd.DataFrame(df_raw)
df_local

,Type,Latitude,Longitude,Message,Date,Time
0,Roadwork,1.290572,103.842833,Roadworks in CTE Tunnel (towards AYE) before H...,27/12/2022,23:57
1,Roadwork,1.306701,103.840467,Roadworks in CTE Tunnel (towards AYE) before O...,27/12/2022,23:55
2,Roadwork,1.396506,103.847941,Roadworks on SLE (towards BKE) after TPE Entra...,27/12/2022,23:54
3,Roadwork,1.326460,103.870421,Roadworks on PIE (towards Tuas) after KPE Exit...,27/12/2022,23:53
4,Roadwork,1.310076,103.870834,Roadworks on Geylang Road (toward City) after ...,27/12/2022,23:50
5,Roadwork,1.328920,103.868459,Roadworks in Woodsville Tunnel (towards Upper ...,27/12/2022,23:48
6,Roadwork,1.293474,103.844128,Roadworks in CTE Tunnel (towards AYE) before H...,27/12/2022,23:45
7,Roadwork,1.353673,103.727579,Roadworks on PIE (towards Tuas) at Jurong East...,27/12/2022,23:45
8,Roadwork,1.341450,103.749688,Roadworks on PIE (towards Changi Airport) at T...,27/12/2022,23:38
9,Roadwork,1.396502,103.904031,Roadworks on TPE (towards SLE) after Punggol R...,27/12/2022,23:28
